### WalkSAT plus Data Analysis of Results, Daniel Espinosa 2024 UCSB ECE Strukov Group + FPGA copy


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.ticker import MultipleLocator,FormatStrFormatter,MaxNLocator
import xml.etree.ElementTree as Xet

In [2]:
import random

def walkSAT(clauses, max_tries, max_flips, p):
    def evaluate_clause(clause, assignment):
        return any((var > 0 and assignment.get(abs(var), False)) or 
                   (var < 0 and not assignment.get(abs(var), False)) for var in clause)

    def get_unsatisfied_clauses(clauses, assignment):
        return [clause for clause in clauses if not evaluate_clause(clause, assignment)]

    def get_variables(clauses):
        return set(abs(var) for clause in clauses for var in clause)

    def flip_variable(assignment, var):
        assignment[var] = not assignment[var]

    for _ in range(max_tries):
        variables = list(get_variables(clauses))
        assignment = {var: random.choice([True, False]) for var in variables}
        
        for _ in range(max_flips):
            unsatisfied = get_unsatisfied_clauses(clauses, assignment)
            if not unsatisfied:
                return assignment  # Found a satisfying assignment
            
            clause = random.choice(unsatisfied)
            if random.random() < p:
                # Flip a random variable from the clause
                var_to_flip = abs(random.choice(clause))
            else:
                # Flip a variable that minimizes the number of unsatisfied clauses if flipped
                break_counts = []
                for var in clause:
                    assignment[abs(var)] = not assignment[abs(var)]
                    break_counts.append((len(get_unsatisfied_clauses(clauses, assignment)), abs(var)))
                    assignment[abs(var)] = not assignment[abs(var)]  # Undo the flip

                min_break = min(break_counts, key=lambda x: x[0])
                vars_with_min_break = [var for break_count, var in break_counts if break_count == min_break[0]]
                var_to_flip = random.choice(vars_with_min_break)
            
            flip_variable(assignment, var_to_flip)

    return "FAIL"

# Example SAT CNF formula: (x1 or not x2) and (not x1 or x2)
# clauses = [[1, -2],[-1,2]]
# Example UNSAT CNF formula: 
clauses = [[1, 2],[-1,-2],[1,-2],[-1,2]]
# Parameters
max_tries = 100
max_flips = 1000
p = 0.3

# Running WalkSAT
result = walkSAT(clauses, max_tries, max_flips, p)
print("Satisfying assignment found:" if result != "FAIL" else "No satisfying assignment found within the given limits.", result)

No satisfying assignment found within the given limits. FAIL


Now, structuring this based on the PDF using binary values for everything

In [3]:
def parse_cnf_file(file_path):
    num_vars = 0
    num_clauses = 0
    clauses = []

    with open(file_path, 'r') as file:
        for line in file:
            # Strip leading and trailing whitespace
            line = line.strip()

            # Ignore comment lines
            if line.startswith('c'):
                continue
            
            # Process the problem line
            elif line.startswith('p'):
                parts = line.split()
                if len(parts) == 4 and parts[1] == 'cnf':
                    num_vars = int(parts[2])
                    num_clauses = int(parts[3])
            
            # Process clause lines
            else:
                # Convert numbers to integers and remove the trailing 0
                clause = [int(x) for x in line.split()[:-1]]
                clauses.append(clause)
    
    return num_vars, num_clauses, clauses

def random_assignment(cnf):
    # cnf.variables is an iterable of variable identifiers
    return {var: random.choice([1, 0]) for var in cnf.variables}

In [115]:
parse_cnf_file('mnci_n16_t3_0_3sat.cnf')

class CNF:
    def __init__(self, num_vars, num_clauses, clauses):
        self.num_vars = num_vars
        self.num_clauses = num_clauses
        self.clauses = clauses

    def __str__(self,verbose=False):
        description = f"CNF Formula with {self.num_vars} variables and {self.num_clauses} clauses:\n"
        if verbose: 
            for clause in self.clauses:
                description += " ".join(str(var) for var in clause) + " 0\n"
        return description

    def variables(self):
        return range(self.num_vars)
    
    def random_assignment(self):
        return [random.choice([0, 1]) for var in CNF.variables(self)]
    
    def variable_membership(self):
        # We are making this so we can quickly make a LUT to see where variables are
        # To update only parts of the CNF and not handle the whole thing at once.
        # Maps each variable to the clauses it appears in (by index)
        """
        Usage: cnf.variable_membership.get(literal_integer) gives us all the clause
        numbers that the literal is a part of. Once we have all the memberships
        we can encode them to a single binary number later.
        """
        membership = {var: [] for var in self.variables()}
        for clause_index, clause in enumerate(self.clauses):
            for var in clause:
                var = abs(var)  # Consider the variable, not its negation
                if var in membership:
                    membership[var].append(clause_index)
        return membership

    def add_clause(self, clause):
        self.clauses.append(clause)
        self.num_clauses += 1

In [116]:
outputs = parse_cnf_file('mnci_n16_t3_0_3sat.cnf')
cnf = CNF(outputs[0],outputs[1],outputs[2])

In [117]:
cnf.__str__(verbose=True)

'CNF Formula with 232 variables and 619 clauses:\n17 13 0\n-17 -13 0\n20 16 0\n-20 -16 0\n25 16 0\n-25 -16 0\n30 15 0\n-30 -15 0\n35 14 0\n-35 -14 0\n42 15 0\n-42 -15 0\n-47 16 0\n47 -16 0\n51 16 0\n-51 -16 0\n57 15 0\n-57 -15 0\n-63 16 0\n63 -16 0\n71 13 0\n-71 -13 0\n75 15 0\n-75 -15 0\n1 2 -31 0\n1 -2 31 0\n-1 2 31 0\n-1 -2 -31 0\n1 3 -21 0\n1 -3 21 0\n-1 3 21 0\n-1 -3 -21 0\n1 5 -52 0\n1 -5 52 0\n-1 5 52 0\n-1 -5 -52 0\n2 3 -43 0\n2 -3 43 0\n-2 3 43 0\n-2 -3 -43 0\n2 5 -26 0\n2 -5 26 0\n-2 5 26 0\n-2 -5 -26 0\n3 7 -18 0\n3 -7 18 0\n-3 7 18 0\n-3 -7 -18 0\n4 8 -48 0\n4 -8 48 0\n-4 8 48 0\n-4 -8 -48 0\n7 9 -17 0\n7 -9 17 0\n-7 9 17 0\n-7 -9 -17 0\n18 12 -19 0\n18 -12 19 0\n-18 12 19 0\n-18 -12 -19 0\n19 14 -20 0\n19 -14 20 0\n-19 14 20 0\n-19 -14 -20 0\n21 5 -22 0\n21 -5 22 0\n-21 5 22 0\n-21 -5 -22 0\n22 9 -23 0\n22 -9 23 0\n-22 9 23 0\n-22 -9 -23 0\n23 10 -24 0\n23 -10 24 0\n-23 10 24 0\n-23 -10 -24 0\n24 15 -25 0\n24 -15 25 0\n-24 15 25 0\n-24 -15 -25 0\n26 6 -27 0\n26 -6 27 0\n-2

In [64]:
cnf.variable_membership().get(1)

[24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 260, 261, 262, 263, 264, 265]

In [66]:
start = cnf.random_assignment() 

In [68]:
start[10]

0

In [72]:
cnf.clauses[0]

[17, 13]

In [107]:
membership_dictionary = cnf.variable_membership()
#[x in range(10), cnf.clauses[membership_dictionary.get(1)[x]]]

start = cnf.random_assignment() 
testclause = cnf.clauses[0]

def evaluate_clause(clause, assignment):
    return any((var > 0 and assignment[abs(var) - 1]) or 
               (var < 0 and not assignment[abs(var) - 1]) for var in clause)

def get_unsatisfied_clauses(clauses, assignment):
    # We are doing this to build the UNSAT buffer for the algorithm
    return [clause for clause in clauses if not evaluate_clause(clause, assignment)]

print(evaluate_clause(testclause, start))
get_unsatisfied_clauses(cnf.clauses, start)

True


[[20, 16],
 [30, 15],
 [35, 14],
 [-47, 16],
 [-18, -12, -19],
 [-19, 14, 20],
 [21, 5, -22],
 [-23, 10, 24],
 [24, 15, -25],
 [26, 6, -27],
 [-29, 14, 30],
 [31, 3, -32],
 [31, 4, -36],
 [31, 5, -58],
 [-32, 4, 64],
 [-32, 5, 72],
 [-32, -7, -33],
 [-33, 8, 34],
 [34, -12, 35],
 [-36, 5, 37],
 [-39, 10, 40],
 [40, -12, 41],
 [41, 13, -42],
 [45, 10, -46],
 [50, 15, -51],
 [52, 6, -53],
 [-54, -11, -55],
 [56, 14, -57],
 [-58, 6, 59],
 [-60, 8, 61],
 [-62, 13, 63],
 [64, 5, -65],
 [68, 10, -69],
 [-69, -11, -70],
 [-70, -12, -71],
 [74, 14, -75],
 [1, 2, -76],
 [3, -79],
 [77, -81],
 [-81],
 [-82, 5, 86],
 [5, -87],
 [-84, -87, -88],
 [-84, -87, 89],
 [85, 89, -90],
 [85, -91],
 [89, -91],
 [-91],
 [86, 6, -92],
 [-88, 93, 94],
 [95, -97],
 [-97],
 [-92, -7, -98],
 [94, -99, 100],
 [-96, 101, 102],
 [101, -103],
 [-103],
 [8, -105],
 [100, -105, 106],
 [100, -107],
 [102, -107, 108],
 [-104, 9, 110],
 [9, -111],
 [108, 113, -114],
 [108, -115],
 [113, -115],
 [109, -115, 116],
 [109, -

In [142]:
break_counts = [(5, 'a'), (3, 'b'), (4, 'c')]
min_break_count = min(break_counts, key=lambda x: x[0])
print(min_break_count[1])  # Output: (3, 'b')

b


In [139]:
# Now, complete method which is an exact copy of what they are doing in the paper.

def evaluate_clause(clause, assignment):
    
    return any((var > 0 and assignment[abs(var) - 1]) or 
               (var < 0 and not assignment[abs(var) - 1]) for var in clause)

def flip_variable(assignment, var):
    #first element is named '1' not '0'
    assignment[var-1] = not assignment[var-1]

def get_unsatisfied_clauses(clauses, assignment):
    #inefficient, parses the entire thing, use at least as possible
    return [clause for clause in clauses if not evaluate_clause(clause, assignment)]

def get_candidate_clauses(clauses, assignment):
    #efficient, only grabs the clauses that are numbered in the membership,use at least as possible

    
def binarywalkSAT(cnf: CNF, max_tries=2, max_flips=1000, p=0.3):
    membership = cnf.variable_membership()
    clauses = cnf.clauses()
    for _ in range(max_tries):
        assignment = cnf.random_assignment()  #1 bit vector assignment
        # unsat clause buffer
        unsatisfied = get_unsatisfied_clauses(clauses, assignment) 

        for _ in range(max_flips):
            if not unsatisfied:
                return assignment  # If all unSat are Sat we are done
            
            clause = random.choice(unsatisfied)

            if random.random() < p:
                var_to_flip = abs(random.choice(clause))
            else:
                break_counts = []
                for var in clause:
                    # Temporary flip (3b in KM10):
                    flip_variable(assignment, abs(var))
                    break_counts.append((len(get_candidate_clauses(clauses, assignment)), abs(var)))
                    flip_variable(assignment, abs(var))  # Undo the flip

                min_break = min(break_counts, key=lambda x: x[0]) #from example above. 
                var_to_flip = min_break[1]
            
            flip_variable(assignment, var_to_flip)
    return "FAIL"


In [136]:
binarywalkSAT(cnf)

'FAIL'

In [138]:
binarywalkSAT(cnf)

TypeError: evaluate_clause() got an unexpected keyword argument 'flip_var'